In [1]:
# setting the N value

N = 4;

# points in t direction
# trigpts returns N equally spaced points on an interval -1 to 1
# range is kind of like linspace! the length

t = pi* range(-1,1, length=N+1)[1:end-1];
t = reshape(t,:,1)


# chebpts returns chebyshev points of the second kind
# generates N+1 cheb pts in the interval [-1, 1], so includes endpoints
# points are clustered near the endpoints

r = [cos((pi*k)/(N-1)) for k in (N-1):-1:0];
z = [cos((pi*k)/(N-1)) for k in (N-1):-1:0];


# going from N-1 to 0 ensures that endpoints are included bc when you 
#plug in (N-1) and 0 into the formula, you get the endpoints included -1 and 1 regardless of N
# cosine part is just a formula for cheb pts of second kind

display(t)
display(z)
display(r)

4×1 reshape(::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, 4, 1) with eltype Float64:
 -3.141592653589793
 -1.5707963267948966
  0.0
  1.5707963267948966

4-element Vector{Float64}:
 -1.0
 -0.4999999999999998
  0.5000000000000001
  1.0

4-element Vector{Float64}:
 -1.0
 -0.4999999999999998
  0.5000000000000001
  1.0

In [2]:
# now to make the grid! this was a bit incorrect because we need the different layers? 
#I found a package that works better!

# tt = 
rr = hcat([r for _ in 1:N]...)
zz = vcat([transpose(z) for _ in 1:N]...)
 
display(rr)
display(zz)

4×4 Matrix{Float64}:
 -1.0  -1.0  -1.0  -1.0
 -0.5  -0.5  -0.5  -0.5
  0.5   0.5   0.5   0.5
  1.0   1.0   1.0   1.0

4×4 Matrix{Float64}:
 -1.0  -0.5  0.5  1.0
 -1.0  -0.5  0.5  1.0
 -1.0  -0.5  0.5  1.0
 -1.0  -0.5  0.5  1.0

In [3]:
# same row vector vertical stacked
rr = [ri for ri in r, zi in z, ti in t]

# same column vector horizontal stacked
zz = [zi for ri in r, zi in z, ti in t]

# every entry in a matrix the same, but each layer has different value
tt = [ti for ri in r, zi in z, ti in t]

display(rr)
display(zz)
display(tt)


4×4×4×1 Array{Float64, 4}:
[:, :, 1, 1] =
 -1.0  -1.0  -1.0  -1.0
 -0.5  -0.5  -0.5  -0.5
  0.5   0.5   0.5   0.5
  1.0   1.0   1.0   1.0

[:, :, 2, 1] =
 -1.0  -1.0  -1.0  -1.0
 -0.5  -0.5  -0.5  -0.5
  0.5   0.5   0.5   0.5
  1.0   1.0   1.0   1.0

[:, :, 3, 1] =
 -1.0  -1.0  -1.0  -1.0
 -0.5  -0.5  -0.5  -0.5
  0.5   0.5   0.5   0.5
  1.0   1.0   1.0   1.0

[:, :, 4, 1] =
 -1.0  -1.0  -1.0  -1.0
 -0.5  -0.5  -0.5  -0.5
  0.5   0.5   0.5   0.5
  1.0   1.0   1.0   1.0

4×4×4×1 Array{Float64, 4}:
[:, :, 1, 1] =
 -1.0  -0.5  0.5  1.0
 -1.0  -0.5  0.5  1.0
 -1.0  -0.5  0.5  1.0
 -1.0  -0.5  0.5  1.0

[:, :, 2, 1] =
 -1.0  -0.5  0.5  1.0
 -1.0  -0.5  0.5  1.0
 -1.0  -0.5  0.5  1.0
 -1.0  -0.5  0.5  1.0

[:, :, 3, 1] =
 -1.0  -0.5  0.5  1.0
 -1.0  -0.5  0.5  1.0
 -1.0  -0.5  0.5  1.0
 -1.0  -0.5  0.5  1.0

[:, :, 4, 1] =
 -1.0  -0.5  0.5  1.0
 -1.0  -0.5  0.5  1.0
 -1.0  -0.5  0.5  1.0
 -1.0  -0.5  0.5  1.0

4×4×4×1 Array{Float64, 4}:
[:, :, 1, 1] =
 -3.14159  -3.14159  -3.14159  -3.14159
 -3.14159  -3.14159  -3.14159  -3.14159
 -3.14159  -3.14159  -3.14159  -3.14159
 -3.14159  -3.14159  -3.14159  -3.14159

[:, :, 2, 1] =
 -1.5708  -1.5708  -1.5708  -1.5708
 -1.5708  -1.5708  -1.5708  -1.5708
 -1.5708  -1.5708  -1.5708  -1.5708
 -1.5708  -1.5708  -1.5708  -1.5708

[:, :, 3, 1] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 4, 1] =
 1.5708  1.5708  1.5708  1.5708
 1.5708  1.5708  1.5708  1.5708
 1.5708  1.5708  1.5708  1.5708
 1.5708  1.5708  1.5708  1.5708

In [9]:
# get the matrices we want: C02, D2, A, coF
using LinearAlgebra, SparseArrays

# C01: Convert Chebyshev-T (T) coefficients to Chebyshev-U (U)
C01 = spdiagm(0 => 0.5*ones(N), 2 => -0.5*ones(N-2))
C01[1,1] = 1.0
println("C01:")
display(C01)
# C12: Convert Chebyshev-U to C^2, the continuous function space
K_1 = 1.0 ./ (1:N)
K_2 = 1.0 ./ (3:N)
C12 = spdiagm(0 => K_1, 2 => -K_2)
println("C12:")
display(C12)
# C02: Convert Chebyshev-T to C^2 (It's C01 * C12)
C02 = C01 * C12
println("C02:")
display(C02)

# D1: Differentiate Chebyshev-T and convert to Chebyshev-U
D1 = spdiagm(1 => 1:N-1)
println("D1: ")
display(D1)
# D2: Differentiate Chebyshev-T twice and convert to C²
D2 = spdiagm(2 => 2 .* (2:N-1))
println("D2: ")
display(D2)
# D0: Differentiate Chebyshev-T once
D0 = C01 \ D1
println("D0: ")
display(D0)

# R0: Multiply Cheb-T by x
R0 = spdiagm(-1 => 0.5*ones(N-1), 1 => 0.5*ones(N-1))
R0[2,1] = 1.0
println("R0:")
display(R0)
# R2: Multiply in C² space
denom = 4:2:2*N+2
K_sub  = ((1:N) ./ denom)[1:N-1]
K_super = ((3:N+2) ./ denom[1:N])[2:N]
R2 = spdiagm(-1 => K_sub, 1 => K_super)
println("R2:")
display(R2)
# R1: Multiply in Cheb-U basis
R1 = C12 \ R2 * C12
println("R1:")
display(R1)

# A: Laplacian operator
A = R2 * R2 * D2 + R2 * C12 * D1
println("A:")
display(A)

# coF: Laplacian multiplier
coF = R2 * R2 * C02
println("coF:")
display(coF)

C01:


4×4 SparseMatrixCSC{Float64, Int64} with 6 stored entries:
 1.0   ⋅   -0.5    ⋅ 
  ⋅   0.5    ⋅   -0.5
  ⋅    ⋅    0.5    ⋅ 
  ⋅    ⋅     ⋅    0.5

C12:


4×4 SparseMatrixCSC{Float64, Int64} with 6 stored entries:
 1.0   ⋅   -0.333333    ⋅ 
  ⋅   0.5    ⋅        -0.25
  ⋅    ⋅    0.333333    ⋅ 
  ⋅    ⋅     ⋅         0.25

C02:


...

4×4 Matrix{Float64}:
 0.0  0.5  0.0  -0.1
 0.5  0.0  0.5   0.0
 0.0  0.5  0.0   0.2
 0.0  0.0  0.5   0.0

A:


4×4 SparseMatrixCSC{Float64, Int64} with 5 stored entries:
  ⋅    ⋅    1.33333    ⋅ 
  ⋅   0.25   ⋅        2.625
  ⋅    ⋅    0.666667   ⋅ 
  ⋅    ⋅     ⋅        1.125

coF:


4×4 SparseMatrixCSC{Float64, Int64} with 8 stored entries:
 0.166667    ⋅       -0.0138889    ⋅ 
  ⋅         0.09375    ⋅         -0.046875
 0.0833333   ⋅        0.0305556    ⋅ 
  ⋅         0.03125    ⋅         -0.003125

In [11]:
# dummy test function
f(r, z, θ, t) = r^2 * sin(θ) * cos(θ) * (1 - r^2) * (1 - z^4)

# initialize CFS array to hold function values at t = 0
CFS = zeros(size(rr))

# Evaluate initial condition on the full (r, z, θ) grid
for i in 1:size(rr, 1)
    for j in 1:size(rr, 2)
        for k in 1:size(rr, 3)
            CFS[i, j, k] = f(rr[i, j, k], zz[i, j, k], tt[i, j, k], 0.0)
        end
    end
end

display(CFS);

4×4×4×1 Array{Float64, 4}:
[:, :, 1, 1] =
 0.0  0.0         0.0         0.0
 0.0  2.1527e-17  2.1527e-17  0.0
 0.0  2.1527e-17  2.1527e-17  0.0
 0.0  0.0         0.0         0.0

[:, :, 2, 1] =
 -0.0  -0.0          -0.0          -0.0
 -0.0  -1.07635e-17  -1.07635e-17  -0.0
 -0.0  -1.07635e-17  -1.07635e-17  -0.0
 -0.0  -0.0          -0.0          -0.0

[:, :, 3, 1] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 4, 1] =
 0.0  0.0          0.0          0.0
 0.0  1.07635e-17  1.07635e-17  0.0
 0.0  1.07635e-17  1.07635e-17  0.0
 0.0  0.0          0.0          0.0

In [ ]:
# replicate V2C_cyl function
using FFTW
using LinearAlgebra

function V2C_cyl(vals::Array, a::AbstractString)
    # julia uses mapslices with dims=d, which works directly on a specific dimension so no need to manually reshape like in MATLAB

    dim = ndims(vals)
    sz = size(vals)
    
    # check that length(a) = # of dimensions
    @assert length(a) == dim "Length of 'a' must match number of dimensions in vals."

    # loop through dims, deciding which transform to apply
    for d in 1:dim
        # if t, FFT along dim
        if a[d] == 't' || a[d] == 'T'
            vals = mapslices(x -> fft(x) / length(x), vals; dims=d)
        elseif a[d] == '0'
            # do nothing
            continue
        # otherwise dct-ii along dim
        else
            vals = mapslices(x -> dct(x, 2), vals; dims=d)
        end

        # julia uses mapslices, which works directly on a specific dimension so no need to reshape and permute dimensions like in MATLAB
    end

    return vals
end